# Building an ITSG-33 RAG chatbot with LangChain, Ollama, and Neo4j

In [50]:
%%sh
pip install langchain langchain-community langchain-experimental json-repair pypdf neo4j

In [1]:
import boto3, json
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.docstore.document import Document


## Connect to Ollama 

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama3:latest",
    base_url='http://ollama-server-2:11434',
    temperature=0,
    top_p=0,
    top_k=40
)  

llm.invoke("Tell me a joke")

"Here's one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!\n\nHope that made you smile! Do you want to hear another one?"

In [3]:
llm_graph_transformer = LLMGraphTransformer(llm=llm)

## Security Control Catalogue
#### Nodes: Control, Control Enhancement, Family, Class, Document, Profile, SuggestedPriority, FunctionalGroup
#### Relationships: PART_OF, BLONGS_TO, REFERENCES, IS_SUGGESTED_FOR, ENHANCES

In [17]:
%%time
NEO4J_URI = 'bolt://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'neo4jgenai'
NEO4J_DATABASE = 'neo4j'

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

CPU times: user 7.5 ms, sys: 0 ns, total: 7.5 ms
Wall time: 2.9 s


In [24]:
%%time
catalogue_query =  """
LOAD CSV WITH HEADERS FROM 'file:///itsg33-ann4a-eng.csv'
AS row

FOREACH (i in CASE WHEN row.enhancement IS NULL OR trim(row.enhancement) = '' THEN [1] ELSE [] END |
        MERGE (ctrl:Control {id:row.family + row.controlId})
        SET ctrl.name = trim(row.name),
            ctrl.definition = trim(row.definition),
            ctrl.supplementalGuidance = trim(row.supplementalGuidance),
            ctrl.genTailorAndImplGuidanceNotes = trim(row.generalTailoringAndImplementationGuidanceNotes),
            ctrl.suggestedPBMMPlaceholderValues = trim(row.suggestedPBMMPlaceholderValues),
            ctrl.PBMMProfileSpecificNotes = trim(row.PBMMProfileSpecificNotes),
            ctrl.suggestedSecretMMPlaceholderValues = trim(row.suggestedSecretMMPlaceholderValues),
            ctrl.SecretMMProfileSpecificNotes = trim(row.SecretMMProfileSpecificNotes)
        MERGE (f:Family {name:row.family})
        MERGE (ctrl)-[:PART_OF]->(f)
        FOREACH (i in CASE WHEN COALESCE(trim(row.class), '') <> '' THEN [1] ELSE [] END |
            MERGE (c:Class {name:row.class})
            MERGE (f)-[:BELONGS_TO]->(c) 
        )        
        FOREACH (i in CASE WHEN COALESCE(trim(row.references), '') <> '' THEN [1] ELSE [] END |
            MERGE (d:Document {name: row.references})
            MERGE (ctrl)-[:REFERENCES]->(d)
        )
        FOREACH (i in CASE WHEN trim(row.suggestedForPBMMProfile) = 'X' THEN [1] ELSE [] END |
                MERGE (pp:Profile {name:'PBMM'})
                MERGE (c)-[:SUGGESTED_FOR]->(pp)
        )
        FOREACH (i in CASE WHEN trim(row.suggestedForSecretMMProfile) = 'X' THEN [1] ELSE [] END |
                MERGE (sp:Profile {name:'SecretMM'})
                MERGE (c)-[:SUGGESTED_FOR]->(sp)
        )
        FOREACH (i in CASE WHEN trim(row.itSecurityFunction)  <> '' THEN [1] ELSE [] END |
                MERGE (itsf:ITSecurity {name:'IT Security Function'})
                FOREACH (i in CASE WHEN trim(row.itSecurityFunction) = 'R' THEN [1] ELSE [] END |
                    MERGE (itsf)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.itSecurityFunction) = 'S' THEN [1] ELSE [] END |
                    MERGE (itsf)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.itOperationsGroup)  <> '' THEN [1] ELSE [] END |
                MERGE (itog:ITOperations {name:'IT Operations Group'})
                FOREACH (i in CASE WHEN trim(row.itOperationsGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (itog)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.itOperationsGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (itog)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.itProjects)  <> '' THEN [1] ELSE [] END |
                MERGE (itp:ITProjects {name:'IT Projects'})
                FOREACH (i in CASE WHEN trim(row.itProjects) = 'R' THEN [1] ELSE [] END |
                    MERGE (itp)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.itProjects) = 'S' THEN [1] ELSE [] END |
                    MERGE (itp)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.physicalSecurityGroup)  <> '' THEN [1] ELSE [] END |
                MERGE (phs:PhysicalSecurity {name:'Physical Security Group'})
                FOREACH (i in CASE WHEN trim(row.physicalSecurityGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (phs)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.physicalSecurityGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (phs)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup)  <> '' THEN [1] ELSE [] END |
                MERGE (ps:PersonnelSecurity {name:'Personnel Security Group'})
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (ps)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (ps)-[:SUPPORTS]->(ctrl)
                )
        )
        FOREACH (i in CASE WHEN trim(row.learningCenter)  <> '' THEN [1] ELSE [] END |
                MERGE (lc:LearningCenter {name:'Learning Center'})
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'R' THEN [1] ELSE [] END |
                    MERGE (lc)-[:RESPONSIBLE_FOR]->(ctrl)
                )
                FOREACH (i in CASE WHEN trim(row.personnelSecurityGroup) = 'S' THEN [1] ELSE [] END |
                    MERGE (lc)-[:SUPPORTS]->(ctrl)
                )                    
        )
        FOREACH (i in CASE 
                        WHEN trim(row.suggestedPriority)  <> '' 
                            AND trim(row.suggestedPriority)  <> 'None defined' 
                        THEN [1] ELSE [] END |
                MERGE (priority:Priority {name:trim(row.suggestedPriority)})
                MERGE (ctrl)-[:HAS_SUGGESTED_PRIORITY]->(priority)
        )
)
"""

kg.query(catalogue_query)


CPU times: user 2.3 ms, sys: 254 µs, total: 2.56 ms
Wall time: 1.09 s


[]